In [1]:
%cd /home/q123/Desktop/explo

import torch 
import gpytorch 
import logging
import logging.config

from src.helpers import setup_experiment
from src.trainer import Trainer
from src.optimizers.gibo import GIBOptimizer
from src.optimizers.vanilla_bo import BOptimizer
from src.config import get_configs

logging.config.fileConfig('logging.conf')
# create root logger
logger = logging.getLogger()
print("hello")


/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/envs/registration.py:415: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
pybullet build time: Jun 23 2022 12:25:14


hello


In [2]:

seed = 21
#env_name = "CartPole-v1" ## Action kernel + State_norm looks very well for cartpole
#env_name = "Swimmer-v4" ##  State_norm stabilizes training 
env_name = "Hopper-v2"
#env_name = "Walker2d-v3"
kernel_name = "rbfstate" ## "linearstate" /"rbfstate"

env_config,likelihood_config,kernel_config,mean_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
env_config["manipulate_state"] = True

optimizer_config = {
        "n_eval":2, ## 3 for cartpole (very noisy)
        ### for GIBO
        "n_max":32, 
        "n_info_samples":16,
        "delta":0.1, ## 0.01 for cartpole
        ### hessian normalisation applies only for rbf
        "normalize_gradient": True,
        "standard_deviation_scaling":False,
}


likelihood_config = {
                "noise_hyperprior":gpytorch.priors.torch_priors.UniformPrior(a=0.01,b=0.05),
                "noise_constraint":gpytorch.constraints.constraints.Interval(0.01,0.05)
                }


kernel_config = {
        "use_ard":True,
        "kernel_name":kernel_name,
        #"lengthscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(2,0.2),
        #"lengthscale_hyperprior":gpytorch.priors.torch_priors.NormalPrior(1,0,),
        "lengthscale_constraint":gpytorch.constraints.constraints.Interval(0.1,10),
        #"outputscale_hyperprior":gpytorch.priors.torch_priors.GammaPrior(2,0.4),
        "outputscale_constraint":gpytorch.constraints.constraints.GreaterThan(0.01),
        }

mean_config = {
                "advantage":True,
        }


policy_config = {
        "add_layer":[],### can be empty or [8,7] for adding 2 layers with width 8,7  neurons respectively
        "add_bias":False,
}

trainer_config = {
        "n_steps":1000, 
        "report_freq":100,
        "save_best":False,
        "wandb_logger":False,
        "run_name" : env_name+kernel_name+str(seed),
        "wandb_config": {**env_config,**optimizer_config,**likelihood_config,**kernel_config,**policy_config}
}


model,objective_env = setup_experiment(env_config,mean_config,kernel_config,likelihood_config,policy_config,
                                seed=seed)


### Chose optimizer 
#optimizer = BOptimizer(**optimizer_config)
optimizer = GIBOptimizer(model,**optimizer_config)
trainer = Trainer(model,objective_env,optimizer,**trainer_config)
#rslt= trainer.run()

### ADD LR SCHEDULAR  / FIX DISCRETIZATION ===> ENJOY WEEKEND :DDD

MathLog.src.helpers : WARNING : MLP dimensions : [11, 3]
Using state normalization
fixing seed to  21
 Gibo will use 32 last points to fit GP and 16 info samples


In [3]:
# for p in objective_env.mlp.parameters():
#     print(p)

In [5]:
input = torch.zeros((133),requires_grad=True)
output = model.mean_module(input)

torch.autograd.grad(output,input,allow_unused=False)


TypeError: 'NoneType' object is not subscriptable

In [ ]:

agent = create_agent(objective_env)
Adv_mean = AdvantageMean(agent,objective_env.mlp)


local_params = torch.rand(1,33)
local_mean,local_states,local_transitions = objective_env.run_many(agent._actor_approximator,5)
print(local_mean)
Adv_mean.set_train_data(local_mean,local_states,local_transitions,local_params)
Adv_mean(local_params)



tensor([0.0179])
local_q torch.Size([94]) q torch.Size([94])


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/mushroom_rl/approximators/parametric/torch_approximator.py:166: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.int)


tensor([0.0179], grad_fn=<AddBackward0>)

In [ ]:
import numpy as np

from mushroom_rl.algorithms.actor_critic.deep_actor_critic import DeepAC
from mushroom_rl.policy import Policy
from mushroom_rl.approximators import Regressor
from mushroom_rl.approximators.parametric import TorchApproximator
from mushroom_rl.utils.replay_memory import ReplayMemory
from mushroom_rl.utils.parameters import Parameter, to_parameter

from copy import deepcopy


class Critic(Object):
    
    
    def __init__(self,state_size,action_size):

        self._critic_fit_params = dict()
        self._critic_predict_params = dict()
        self._replay_memory = ReplayMemory(initial_replay_size, max_replay_size)

        target_critic_params = deepcopy(critic_params)
        self._critic_approximator = Regressor(TorchApproximator,
                                                **critic_params)
        self._target_critic_approximator = Regressor(TorchApproximator,
                                                        **target_critic_params)
        
        
    

        self._init_target(self._critic_approximator,
                          self._target_critic_approximator)
        self._init_target(self._actor_approximator,
                          self._target_actor_approximator)



        

NameError: name 'Object' is not defined

In [ ]:
import torch.nn.functional as F

weight = torch.rand(4,4,4)
input = torch.rand(4)
rslt = F.linear(input=input, weight=weight)
rslt

RuntimeError: t() expects a tensor with <= 2 dimensions, but self is 3D

In [ ]:
# import scipy.stats as stats 
# import numpy as np
# import matplotlib.pyplot as plt

# alpha = 2
# beta = 0.3
# x = np.linspace (0, 10, 200) 
# y1 = stats.gamma.pdf(x, a=alpha, scale=1/beta)
# plt.plot(x,y1)